!pip install pyspark
!pip install pyarrow
!pip install ydata_profiling
!pip install findspark
!pip install numpy --upgrade

In [2]:
import findspark
findspark.init() 

In [3]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [4]:
spark = SparkSession.builder.getOrCreate()

In [10]:
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "storage.yandexcloud.net")
spark._jsc.hadoopConfiguration().set("fs.s3a.signing-algorithm", "")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key","YCAJEWkIYLGko_-k3dtUorfwG")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key","YCPC7oQalSOZW_KDsXGiGJnjB5KJcq8-YITNzErf")

In [22]:
import transformers

In [23]:
df = spark.read.option('comment', '#').\
                    option('timestampFormat', 'yyyy-MM-dd HH:mm:ss').\
                    schema("tranaction_id LONG, tx_datetime STRING, customer_id INT, terminal_id INT, tx_amount DOUBLE, tx_time_seconds LONG, tx_time_days LONG, tx_fraud INT, tx_fraud_scenario INT").\
                    format('csv').\
                    load(f's3a://bucket-mlops-fraud-system/*.txt')
data_filter = transformers.DataFilter()

In [24]:
transformed_df = data_filter.transform(df)

In [ ]:
transformed_df.write.parquet("s3a://bucket-mlops-fraud-system/data_clean.parquet",mode="overwrite")